In [1]:
#!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py

In [2]:
import minsearch

In [3]:
import json

In [4]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)


In [5]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
  
        doc['course'] = course_dict['course']
        documents.append(doc)

documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [6]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [7]:
q = 'the course has already started, can I still enroll?'

In [8]:
index.fit(documents)

In [9]:
boost = {
    "question": 3.0,
    "section": 0.5
}

res = index.search(
    query = q,
    filter_dict={'course': 'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results = 5
)
res

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [10]:
from google import genai
import os
api_key = os.getenv("GEMINI_API_KEY") #key saved as a secret
client = genai.Client(api_key=api_key)

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents = 'the course has already started, can I still enroll?'
)

print(response.text)

Whether or not you can still enroll in a course that has already started depends on several factors:

*   **The Course Provider:** Different institutions have different policies. Some are very strict about enrollment deadlines, while others are more flexible.
*   **The Course Type:** Some courses, like self-paced online courses, often allow enrollment at any time. Others, like university courses with specific start and end dates, might have stricter deadlines.
*   **The Course Length:** If the course is very short (e.g., a week-long workshop), it's unlikely you'll be able to enroll late. If it's a longer course (e.g., a semester-long college class), there might be a window for late enrollment.
*   **Availability of Materials:** If the course involves access to physical materials, enrolling late might mean missing out on those.
*   **The Instructor's Permission:** In some cases, you might be able to enroll late with the instructor's approval. They may want to assess your ability to catc

In [11]:
prompt_template = """ 
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. Use only the facts from the CONTEXT when answering the question.
If the context does not contain the answer, output NONE

QUESTION: {question}
CONTEXT: {context}
""".strip()

context = ""

for doc in res:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
context

In [12]:
context = ""

for doc in res:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
context

"section: General course-related questions\nquestion: Course - Can I still join the course after the start date?\nanswer: Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.\n\nsection: General course-related questions\nquestion: Course - Can I follow the course after it finishes?\nanswer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.\n\nsection: General course-related questions\nquestion: Course - When will the course start?\nanswer: The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course 

In [13]:
prompt = prompt_template.format(question=q, context=context).strip()

In [14]:
prompt 

"You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. Use only the facts from the CONTEXT when answering the question.\nIf the context does not contain the answer, output NONE\n\nQUESTION: the course has already started, can I still enroll?\nCONTEXT: section: General course-related questions\nquestion: Course - Can I still join the course after the start date?\nanswer: Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.\n\nsection: General course-related questions\nquestion: Course - Can I follow the course after it finishes?\nanswer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start workin

In [15]:
from google import genai
import os
api_key = os.getenv("GEMINI_API_KEY") #key saved as a secret
client = genai.Client(api_key=api_key)

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents = prompt 
)

print(response.text)

Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects, so don't leave everything for the last minute.



In [16]:
def search(query):
    boost = {
    "question": 3.0,
    "section": 0.5
    }
    
    res = index.search(
        query = query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results = 5
    )
    return res

In [29]:
def build_prompt(query, search_results):
    prompt_template = """ 
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. Use only the facts from the CONTEXT when answering the question.
If the context does not contain the answer, output NONE

QUESTION: {question}
CONTEXT: {context}
""".strip()

    context = ""
    
    for doc in res:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    prompt = prompt_template.format(question=q, context=context).strip()
    return prompt

In [38]:
def llm(prompt):
    api_key = os.getenv("GEMINI_API_KEY") #key saved as a secret
    client = genai.Client(api_key=api_key)
    
    response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents = prompt )
    return response.text

In [39]:
query = "what is an api"
search_results = search(query)
prompt = build_prompt("what is an api",search_results)
answer = llm(prompt)

In [40]:
prompt

"You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. Use only the facts from the CONTEXT when answering the question.\nIf the context does not contain the answer, output NONE\n\nQUESTION: the course has already started, can I still enroll?\nCONTEXT: section: General course-related questions\nquestion: Course - Can I still join the course after the start date?\nanswer: Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.\n\nsection: General course-related questions\nquestion: Course - Can I follow the course after it finishes?\nanswer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start workin